In [9]:
# Dataframe from NSE Option Chains

# STATUS: Complete

import requests
import lxml.html as lh
import datetime as dt
import pandas as pd
import numpy as np

symbol = 'ACC'  # Sample symbol

url_base = "https://nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol="
url_end = '&date='
url = url_base + symbol

page = requests.get(url)
doc = lh.fromstring(page.content)

#...    Get the Expiries ...#
#____________________________

the_path= "//form"   # Get the form content. There are two forms
expiries = doc.xpath(the_path)[0].text_content()
strip_chars = [ord('\n'), ord('\xa0'), ord('\t'), ord('\r')] # characters to be stripped
char_table = {s: ' ' for s in strip_chars} # table for translate to locate the chars

sym_exp = expiries.translate(char_table).split() # split converts the translated dict into list

# Get the expiries only, from the table
expiry = [sym_exp[k+i] 
 for k, v in enumerate(sym_exp) 
 if v in 'Select' 
 for i in range(len(sym_exp) - k)][1:]

# Convert expiry to datetime
expiry_dt = [dt.datetime.strptime(date, "%d%b%Y").date() for date in expiry]

#...   Get the Options Data .... #
#_________________________________

tbl_path = "//*[@id='octable']"  # xpath for the table

# The headers
opt_head = ['cOI', 'cChnginOI', 'cVolume', 'cIV', 'cLTP', 
            'cNetChng', 'cBidQty', 'cBidPrice', 'cAskPrice', 'cAskQty', 'Strike', 
            'pBidQty', 'pBidPrice', 'pAskPrice', 'pAskQty', 'pNetChng', 'pLTP', 
            'pIV', 'pVolume', 'pChnginOI', 'pOI']

opt_data = [tr.text_content() for table in doc.xpath(tbl_path) for tr in table[1:]]

df1 = pd.DataFrame([d.split() for d in opt_data], columns=opt_head)

df2 = df1.replace(',', '', regex=True) # Remove comma from numbers
df2 = df2.apply(pd.to_numeric, errors='coerce') # Convert to numeric
df2.insert(0, 'Expiry', expiry_dt[1]) # Insert the Expiry column
df2.insert(0, 'nseSymbol', symbol) # Insert the Symbol

# Get the underlying's price
und_price = float(str(pd.read_html(page.text)[0][1]).split(' ')[7])
df2.insert(loc=0, column='undPrice', value=und_price)

# Rearrange the columns
cols_beginning = ['nseSymbol', 'Expiry', 'Strike', 'undPrice']
df2 = df2[cols_beginning + [c for c in df2 if c not in cols_beginning]]

In [10]:
expiry_dt

[datetime.date(2018, 11, 29), datetime.date(2018, 12, 27)]